# NHL Player Analysis

## Notebook Setup

To import the required python libraries and setting notebook display parameters.

In [74]:
# Import libraries
import pandas as pd
import numpy as np
import os.path as path
from sklearn import preprocessing
from sklearn.metrics import mean_squared_error, mean_absolute_error
import warnings
import math
import glob
import os
warnings.simplefilter(action = 'ignore', category = FutureWarning)
warnings.simplefilter(action = 'ignore', category = UserWarning)

import tensorflow as tf
np.random.seed(0)
tf.random.set_seed(0)

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import SimpleRNN, Activation, Dropout, Dense, Reshape, LSTM, GRU

In [51]:
# Settings to allow all columns in dataframe to display
desired_width = 320
display_columns = 222
pd.set_option('display.width', desired_width)
np.set_printoptions(linewidth = desired_width)
pd.set_option('display.max_columns', display_columns)

## Loading Data

Loading NHL player data for multiple years into one master dataset.<br>
Source: https://www.moneypuck.com

In [52]:
def load_csv(folder_path):
    """Load multiple .csv files from a specific folder path"""
    
    # Return a list of paths matching a pathname pattern.
    all_files = glob.glob(folder_path + "/*.csv")

    # Initiate an empty list to store the data
    li = []

    # Loop to append each files data to the list
    for filename in all_files:
        df = pd.read_csv(filename, index_col=None, header=0)
        li.append(df)

    # Join all the files into one DataFrame
    df = pd.concat(li, axis=0, ignore_index=True)

    df = df.sort_values(['name','season'], ascending=False)
    print('The shape of the data is:', df.shape)
    
    return df

In [53]:
# Local path to folder containing files for each year
path = '/Users/nathananderson/Documents/Data_Science/NHL_Player_Prediction/git/NHL_Player_Prediction/Datasets/nhl_player_data'

df = load_csv(path)
df.head()

The shape of the data is: (49495, 154)


,playerId,season,name,team,position,situation,games_played,icetime,shifts,gameScore,onIce_xGoalsPercentage,offIce_xGoalsPercentage,onIce_corsiPercentage,offIce_corsiPercentage,onIce_fenwickPercentage,offIce_fenwickPercentage,iceTimeRank,I_F_xOnGoal,I_F_xGoals,I_F_xRebounds,I_F_xFreeze,I_F_xPlayStopped,I_F_xPlayContinuedInZone,I_F_xPlayContinuedOutsideZone,I_F_flurryAdjustedxGoals,I_F_scoreVenueAdjustedxGoals,I_F_flurryScoreVenueAdjustedxGoals,I_F_primaryAssists,I_F_secondaryAssists,I_F_shotsOnGoal,I_F_missedShots,I_F_blockedShotAttempts,I_F_shotAttempts,I_F_points,I_F_goals,I_F_rebounds,I_F_reboundGoals,I_F_freeze,I_F_playStopped,I_F_playContinuedInZone,I_F_playContinuedOutsideZone,I_F_savedShotsOnGoal,I_F_savedUnblockedShotAttempts,penalties,I_F_penalityMinutes,I_F_faceOffsWon,I_F_hits,I_F_takeaways,I_F_giveaways,I_F_lowDangerShots,I_F_mediumDangerShots,I_F_highDangerShots,I_F_lowDangerxGoals,I_F_mediumDangerxGoals,I_F_highDangerxGoals,I_F_lowDangerGoals,I_F_mediumDangerGoals,I_F_highDangerGoals,I_F_scoreAdjustedShotsAttempts,I_F_unblockedShotAttempts,I_F_scoreAdjustedUnblockedShotAttempts,I_F_dZoneGiveaways,I_F_xGoalsFromxReboundsOfShots,I_F_xGoalsFromActualReboundsOfShots,I_F_reboundxGoals,I_F_xGoals_with_earned_rebounds,I_F_xGoals_with_earned_rebounds_scoreAdjusted,I_F_xGoals_with_earned_rebounds_scoreFlurryAdjusted,I_F_shifts,I_F_oZoneShiftStarts,I_F_dZoneShiftStarts,I_F_neutralZoneShiftStarts,I_F_flyShiftStarts,I_F_oZoneShiftEnds,I_F_dZoneShiftEnds,I_F_neutralZoneShiftEnds,I_F_flyShiftEnds,faceoffsWon,faceoffsLost,timeOnBench,penalityMinutes,penalityMinutesDrawn,penaltiesDrawn,shotsBlockedByPlayer,OnIce_F_xOnGoal,OnIce_F_xGoals,OnIce_F_flurryAdjustedxGoals,OnIce_F_scoreVenueAdjustedxGoals,OnIce_F_flurryScoreVenueAdjustedxGoals,OnIce_F_shotsOnGoal,OnIce_F_missedShots,OnIce_F_blockedShotAttempts,OnIce_F_shotAttempts,OnIce_F_goals,OnIce_F_rebounds,OnIce_F_reboundGoals,OnIce_F_lowDangerShots,OnIce_F_mediumDangerShots,OnIce_F_highDangerShots,OnIce_F_lowDangerxGoals,OnIce_F_mediumDangerxGoals,OnIce_F_highDangerxGoals,OnIce_F_lowDangerGoals,OnIce_F_mediumDangerGoals,OnIce_F_highDangerGoals,OnIce_F_scoreAdjustedShotsAttempts,OnIce_F_unblockedShotAttempts,OnIce_F_scoreAdjustedUnblockedShotAttempts,OnIce_F_xGoalsFromxReboundsOfShots,OnIce_F_xGoalsFromActualReboundsOfShots,OnIce_F_reboundxGoals,OnIce_F_xGoals_with_earned_rebounds,OnIce_F_xGoals_with_earned_rebounds_scoreAdjusted,OnIce_F_xGoals_with_earned_rebounds_scoreFlurryAdjusted,OnIce_A_xOnGoal,OnIce_A_xGoals,OnIce_A_flurryAdjustedxGoals,OnIce_A_scoreVenueAdjustedxGoals,OnIce_A_flurryScoreVenueAdjustedxGoals,OnIce_A_shotsOnGoal,OnIce_A_missedShots,OnIce_A_blockedShotAttempts,OnIce_A_shotAttempts,OnIce_A_goals,OnIce_A_rebounds,OnIce_A_reboundGoals,OnIce_A_lowDangerShots,OnIce_A_mediumDangerShots,OnIce_A_highDangerShots,OnIce_A_lowDangerxGoals,OnIce_A_mediumDangerxGoals,OnIce_A_highDangerxGoals,OnIce_A_lowDangerGoals,OnIce_A_mediumDangerGoals,OnIce_A_highDangerGoals,OnIce_A_scoreAdjustedShotsAttempts,OnIce_A_unblockedShotAttempts,OnIce_A_scoreAdjustedUnblockedShotAttempts,OnIce_A_xGoalsFromxReboundsOfShots,OnIce_A_xGoalsFromActualReboundsOfShots,OnIce_A_reboundxGoals,OnIce_A_xGoals_with_earned_rebounds,OnIce_A_xGoals_with_earned_rebounds_scoreAdjusted,OnIce_A_xGoals_with_earned_rebounds_scoreFlurryAdjusted,OffIce_F_xGoals,OffIce_A_xGoals,OffIce_F_shotAttempts,OffIce_A_shotAttempts,xGoalsForAfterShifts,xGoalsAgainstAfterShifts,corsiForAfterShifts,corsiAgainstAfterShifts,fenwickForAfterShifts,fenwickAgainstAfterShifts
21780,8469820,2013,Zenon Konopka,BUF,C,other,59,307.0,9.0,-0.40,0.14,0.32,0.47,0.49,0.36,0.46,82.0,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,6.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00,0.00,0.0,0.0,0.0,0.0,0.00,0.0,0.00,0.0,0.00,0.00,0.0,0.00,0.00,0.00,9.0,2.0,6.0,0.0,1.0,0.0,0.0,1.0,8.0,6.0,6.0,1790.0,0.0,0.0,0.0,0.0,2.99,0.15,0.13,0.15,0.13,1.0,3.0,3.0,7.0,0.0,1.0,0.0,4.0,0.0,0.0,0.15,0.00,0.00,0.

## Feature Engineering

To create and add new feature columns to the master DataFrame.

In [54]:
def new_features(df):
    """Add new feature columns to the DataFrame."""
    
    # New column for last season played
    df['Last_Season_Played'] = df.groupby(["playerId"])['season'].transform(np.max)
    
    df["% Games Played"] = df['games_played'] / 82
    df["Points per Game"] = df['I_F_points'] / df['games_played']
    df["Goals per Game"] = df['I_F_goals'] / df['games_played']
    df["Shots on Goal per Game"] = df['I_F_shotsOnGoal'] / df['games_played']
    #player_data["Primary Assists per Game"] = player_data['I_F_primaryAssists'] / player_data['games_played']
    #player_data["Secondary Assists per Game"] = player_data['I_F_secondaryAssists'] / player_data['games_played']
        
    return df

In [55]:
df = new_features(df)
df.head()

,playerId,season,name,team,position,situation,games_played,icetime,shifts,gameScore,onIce_xGoalsPercentage,offIce_xGoalsPercentage,onIce_corsiPercentage,offIce_corsiPercentage,onIce_fenwickPercentage,offIce_fenwickPercentage,iceTimeRank,I_F_xOnGoal,I_F_xGoals,I_F_xRebounds,I_F_xFreeze,I_F_xPlayStopped,I_F_xPlayContinuedInZone,I_F_xPlayContinuedOutsideZone,I_F_flurryAdjustedxGoals,I_F_scoreVenueAdjustedxGoals,I_F_flurryScoreVenueAdjustedxGoals,I_F_primaryAssists,I_F_secondaryAssists,I_F_shotsOnGoal,I_F_missedShots,I_F_blockedShotAttempts,I_F_shotAttempts,I_F_points,I_F_goals,I_F_rebounds,I_F_reboundGoals,I_F_freeze,I_F_playStopped,I_F_playContinuedInZone,I_F_playContinuedOutsideZone,I_F_savedShotsOnGoal,I_F_savedUnblockedShotAttempts,penalties,I_F_penalityMinutes,I_F_faceOffsWon,I_F_hits,I_F_takeaways,I_F_giveaways,I_F_lowDangerShots,I_F_mediumDangerShots,I_F_highDangerShots,I_F_lowDangerxGoals,I_F_mediumDangerxGoals,I_F_highDangerxGoals,I_F_lowDangerGoals,I_F_mediumDangerGoals,I_F_highDangerGoals,I_F_scoreAdjustedShotsAttempts,I_F_unblockedShotAttempts,I_F_scoreAdjustedUnblockedShotAttempts,I_F_dZoneGiveaways,I_F_xGoalsFromxReboundsOfShots,I_F_xGoalsFromActualReboundsOfShots,I_F_reboundxGoals,I_F_xGoals_with_earned_rebounds,I_F_xGoals_with_earned_rebounds_scoreAdjusted,I_F_xGoals_with_earned_rebounds_scoreFlurryAdjusted,I_F_shifts,I_F_oZoneShiftStarts,I_F_dZoneShiftStarts,I_F_neutralZoneShiftStarts,I_F_flyShiftStarts,I_F_oZoneShiftEnds,I_F_dZoneShiftEnds,I_F_neutralZoneShiftEnds,I_F_flyShiftEnds,faceoffsWon,faceoffsLost,timeOnBench,penalityMinutes,penalityMinutesDrawn,penaltiesDrawn,shotsBlockedByPlayer,OnIce_F_xOnGoal,OnIce_F_xGoals,OnIce_F_flurryAdjustedxGoals,OnIce_F_scoreVenueAdjustedxGoals,OnIce_F_flurryScoreVenueAdjustedxGoals,OnIce_F_shotsOnGoal,OnIce_F_missedShots,OnIce_F_blockedShotAttempts,OnIce_F_shotAttempts,OnIce_F_goals,OnIce_F_rebounds,OnIce_F_reboundGoals,OnIce_F_lowDangerShots,OnIce_F_mediumDangerShots,OnIce_F_highDangerShots,OnIce_F_lowDangerxGoals,OnIce_F_mediumDangerxGoals,OnIce_F_highDangerxGoals,OnIce_F_lowDangerGoals,OnIce_F_mediumDangerGoals,OnIce_F_highDangerGoals,OnIce_F_scoreAdjustedShotsAttempts,OnIce_F_unblockedShotAttempts,OnIce_F_scoreAdjustedUnblockedShotAttempts,OnIce_F_xGoalsFromxReboundsOfShots,OnIce_F_xGoalsFromActualReboundsOfShots,OnIce_F_reboundxGoals,OnIce_F_xGoals_with_earned_rebounds,OnIce_F_xGoals_with_earned_rebounds_scoreAdjusted,OnIce_F_xGoals_with_earned_rebounds_scoreFlurryAdjusted,OnIce_A_xOnGoal,OnIce_A_xGoals,OnIce_A_flurryAdjustedxGoals,OnIce_A_scoreVenueAdjustedxGoals,OnIce_A_flurryScoreVenueAdjustedxGoals,OnIce_A_shotsOnGoal,OnIce_A_missedShots,OnIce_A_blockedShotAttempts,OnIce_A_shotAttempts,OnIce_A_goals,OnIce_A_rebounds,OnIce_A_reboundGoals,OnIce_A_lowDangerShots,OnIce_A_mediumDangerShots,OnIce_A_highDangerShots,OnIce_A_lowDangerxGoals,OnIce_A_mediumDangerxGoals,OnIce_A_highDangerxGoals,OnIce_A_lowDangerGoals,OnIce_A_mediumDangerGoals,OnIce_A_highDangerGoals,OnIce_A_scoreAdjustedShotsAttempts,OnIce_A_unblockedShotAttempts,OnIce_A_scoreAdjustedUnblockedShotAttempts,OnIce_A_xGoalsFromxReboundsOfShots,OnIce_A_xGoalsFromActualReboundsOfShots,OnIce_A_reboundxGoals,OnIce_A_xGoals_with_earned_rebounds,OnIce_A_xGoals_with_earned_rebounds_scoreAdjusted,OnIce_A_xGoals_with_earned_rebounds_scoreFlurryAdjusted,OffIce_F_xGoals,OffIce_A_xGoals,OffIce_F_shotAttempts,OffIce_A_shotAttempts,xGoalsForAfterShifts,xGoalsAgainstAfterShifts,corsiForAfterShifts,corsiAgainstAfterShifts,fenwickForAfterShifts,fenwickAgainstAfterShifts,Last_Season_Played,% Games Played,Points per Game,Goals per Game,Shots on Goal per Game
21780,8469820,2013,Zenon Konopka,BUF,C,other,59,307.0,9.0,-0.40,0.14,0.32,0.47,0.49,0.36,0.46,82.0,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,6.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00,0.00,0.0,0.0,0.0,0.0,0.00,0.0,0.00,0.0,0.00,0.00,0.0,0.00,0.00,0.00,9.0,2.0,6.0,0.0,1.0,0.0,0.0,1.0,8.0,6.0,6.0,1790.0,0.0,0.0,0.

## Slicing DataFrame

To slice the DataFrame based on selected criteria.

In [56]:
def slice_df(df):
    """Slice the DataFrame based on specific criteria."""

    df = df[df['situation'] == 'all']
    #df = df[df['I_F_points'] >= 10]
    df = df[df['Last_Season_Played'] ==  df['season'].max()]

    return df

In [57]:
df = slice_df(df)
df.head()

,playerId,season,name,team,position,situation,games_played,icetime,shifts,gameScore,onIce_xGoalsPercentage,offIce_xGoalsPercentage,onIce_corsiPercentage,offIce_corsiPercentage,onIce_fenwickPercentage,offIce_fenwickPercentage,iceTimeRank,I_F_xOnGoal,I_F_xGoals,I_F_xRebounds,I_F_xFreeze,I_F_xPlayStopped,I_F_xPlayContinuedInZone,I_F_xPlayContinuedOutsideZone,I_F_flurryAdjustedxGoals,I_F_scoreVenueAdjustedxGoals,I_F_flurryScoreVenueAdjustedxGoals,I_F_primaryAssists,I_F_secondaryAssists,I_F_shotsOnGoal,I_F_missedShots,I_F_blockedShotAttempts,I_F_shotAttempts,I_F_points,I_F_goals,I_F_rebounds,I_F_reboundGoals,I_F_freeze,I_F_playStopped,I_F_playContinuedInZone,I_F_playContinuedOutsideZone,I_F_savedShotsOnGoal,I_F_savedUnblockedShotAttempts,penalties,I_F_penalityMinutes,I_F_faceOffsWon,I_F_hits,I_F_takeaways,I_F_giveaways,I_F_lowDangerShots,I_F_mediumDangerShots,I_F_highDangerShots,I_F_lowDangerxGoals,I_F_mediumDangerxGoals,I_F_highDangerxGoals,I_F_lowDangerGoals,I_F_mediumDangerGoals,I_F_highDangerGoals,I_F_scoreAdjustedShotsAttempts,I_F_unblockedShotAttempts,I_F_scoreAdjustedUnblockedShotAttempts,I_F_dZoneGiveaways,I_F_xGoalsFromxReboundsOfShots,I_F_xGoalsFromActualReboundsOfShots,I_F_reboundxGoals,I_F_xGoals_with_earned_rebounds,I_F_xGoals_with_earned_rebounds_scoreAdjusted,I_F_xGoals_with_earned_rebounds_scoreFlurryAdjusted,I_F_shifts,I_F_oZoneShiftStarts,I_F_dZoneShiftStarts,I_F_neutralZoneShiftStarts,I_F_flyShiftStarts,I_F_oZoneShiftEnds,I_F_dZoneShiftEnds,I_F_neutralZoneShiftEnds,I_F_flyShiftEnds,faceoffsWon,faceoffsLost,timeOnBench,penalityMinutes,penalityMinutesDrawn,penaltiesDrawn,shotsBlockedByPlayer,OnIce_F_xOnGoal,OnIce_F_xGoals,OnIce_F_flurryAdjustedxGoals,OnIce_F_scoreVenueAdjustedxGoals,OnIce_F_flurryScoreVenueAdjustedxGoals,OnIce_F_shotsOnGoal,OnIce_F_missedShots,OnIce_F_blockedShotAttempts,OnIce_F_shotAttempts,OnIce_F_goals,OnIce_F_rebounds,OnIce_F_reboundGoals,OnIce_F_lowDangerShots,OnIce_F_mediumDangerShots,OnIce_F_highDangerShots,OnIce_F_lowDangerxGoals,OnIce_F_mediumDangerxGoals,OnIce_F_highDangerxGoals,OnIce_F_lowDangerGoals,OnIce_F_mediumDangerGoals,OnIce_F_highDangerGoals,OnIce_F_scoreAdjustedShotsAttempts,OnIce_F_unblockedShotAttempts,OnIce_F_scoreAdjustedUnblockedShotAttempts,OnIce_F_xGoalsFromxReboundsOfShots,OnIce_F_xGoalsFromActualReboundsOfShots,OnIce_F_reboundxGoals,OnIce_F_xGoals_with_earned_rebounds,OnIce_F_xGoals_with_earned_rebounds_scoreAdjusted,OnIce_F_xGoals_with_earned_rebounds_scoreFlurryAdjusted,OnIce_A_xOnGoal,OnIce_A_xGoals,OnIce_A_flurryAdjustedxGoals,OnIce_A_scoreVenueAdjustedxGoals,OnIce_A_flurryScoreVenueAdjustedxGoals,OnIce_A_shotsOnGoal,OnIce_A_missedShots,OnIce_A_blockedShotAttempts,OnIce_A_shotAttempts,OnIce_A_goals,OnIce_A_rebounds,OnIce_A_reboundGoals,OnIce_A_lowDangerShots,OnIce_A_mediumDangerShots,OnIce_A_highDangerShots,OnIce_A_lowDangerxGoals,OnIce_A_mediumDangerxGoals,OnIce_A_highDangerxGoals,OnIce_A_lowDangerGoals,OnIce_A_mediumDangerGoals,OnIce_A_highDangerGoals,OnIce_A_scoreAdjustedShotsAttempts,OnIce_A_unblockedShotAttempts,OnIce_A_scoreAdjustedUnblockedShotAttempts,OnIce_A_xGoalsFromxReboundsOfShots,OnIce_A_xGoalsFromActualReboundsOfShots,OnIce_A_reboundxGoals,OnIce_A_xGoals_with_earned_rebounds,OnIce_A_xGoals_with_earned_rebounds_scoreAdjusted,OnIce_A_xGoals_with_earned_rebounds_scoreFlurryAdjusted,OffIce_F_xGoals,OffIce_A_xGoals,OffIce_F_shotAttempts,OffIce_A_shotAttempts,xGoalsForAfterShifts,xGoalsAgainstAfterShifts,corsiForAfterShifts,corsiAgainstAfterShifts,fenwickForAfterShifts,fenwickAgainstAfterShifts,Last_Season_Played,% Games Played,Points per Game,Goals per Game,Shots on Goal per Game
8171,8476878,2022,Zemgus Girgensons,BUF,L,all,72,55559.0,1316.0,22.67,0.44,0.50,0.46,0.52,0.45,0.51,674.0,102.81,11.02,6.86,21.77,3.34,55.69,43.31,10.76,10.94,10.69,4.0,3.0,94.0,48.0,30.0,172.0,16.0,9.0,8.0,1.0,12.0,2.0,45.0,66.0,85.0,133.0,7.0,14.0,88.0,91.0,19.0,7.0,92.0,41.0,9.0,2.89,4.74,3.39,2.0,4.0,3.0,170.78,142.0,140.91,3.0,1.44,2.02,1.54,10.93,10.89,10.72,1316.0,149.0,252

## PlayerID DataFrame

To get a DataFrame of players names and IDs to be used as a lookup table.

In [58]:
def playerid_df(df):
    """DataFrame for player names and IDs."""
    
    # Player Name & Player ID DataFrame
    players = df[['name', 'playerId','Last_Season_Played']].copy()
    
    # Removing duplicates
    players = players.drop_duplicates(subset=['playerId'], keep='first')
    players = players.sort_values('playerId').reset_index(drop = True)
    
    return players

In [59]:
players = playerid_df(df)
players.head()

,name,playerId,Last_Season_Played
0,Eric Staal,8470595,2022
1,Ryan Suter,8470600,2022
2,Jeff Carter,8470604,2022
3,Zach Parise,8470610,2022
4,Brent Burns,8470613,2022


## Machine Learning DataFrame

Create a DataFrame that is formatted to be used with various ML algorithms.

In [60]:
# DataFrame for ML
ml_df = df[['playerId', 'position', 'icetime',
            '% Games Played', 'Points per Game', 'Shots on Goal per Game',
            'Goals per Game', 'Shots on Goal per Game'
          ]].copy()

ml_df.head()

,playerId,position,icetime,% Games Played,Points per Game,Shots on Goal per Game,Goals per Game,Shots on Goal per Game
8171,8476878,L,55559.0,0.878049,0.222222,1.305556,0.125000,1.305556
27636,8476878,L,49692.0,0.682927,0.321429,1.642857,0.178571,1.642857
22871,8476878,C,57145.0,0.841463,0.275362,1.217391,0.173913,1.217391
35606,8476878,C,58768.0,0.878049,0.250000,1.083333,0.069444,1.083333
191,8476878,C,58404.0,0.865854,0.211268,1.591549,0.098592,1.591549


In [61]:
# Instantiate MinMaxScaler
from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler()

# Normalizing specific column so the values are between 0 and 1
ml_df[['icetime']] = scaler.fit_transform(ml_df[['icetime']])
ml_df[['Shots on Goal per Game']] = scaler.fit_transform(ml_df[['Shots on Goal per Game']])
ml_df.head()

,playerId,position,icetime,% Games Played,Points per Game,Shots on Goal per Game,Goals per Game,Shots on Goal per Game
8171,8476878,L,0.383599,0.878049,0.222222,0.256231,0.125000,0.256231
27636,8476878,L,0.343013,0.682927,0.321429,0.322430,0.178571,0.322430
22871,8476878,C,0.394570,0.841463,0.275362,0.238927,0.173913,0.238927
35606,8476878,C,0.405797,0.878049,0.250000,0.212617,0.069444,0.212617
191,8476878,C,0.403279,0.865854,0.211268,0.312360,0.098592,0.312360


In [62]:
"""
# OneHotEncode categorical columns
# OneHotEncode 'position'
onehot_position = pd.get_dummies(ml_df['position'])
ml_df = ml_df.join(onehot_position)
ml_df = ml_df.drop('position', axis=1)

# Rename onehot_position column names
pos_cols = {'C': 'Position - C',
            'D': 'Position - D',
            'L': 'Position - L',
            'R': 'Position - R'}

ml_df.rename(columns=pos_cols, inplace=True)

ml_df.head()
"""

"\n# OneHotEncode categorical columns\n# OneHotEncode 'position'\nonehot_position = pd.get_dummies(ml_df['position'])\nml_df = ml_df.join(onehot_position)\nml_df = ml_df.drop('position', axis=1)\n\n# Rename onehot_position column names\npos_cols = {'C': 'Position - C',\n            'D': 'Position - D',\n            'L': 'Position - L',\n            'R': 'Position - R'}\n\nml_df.rename(columns=pos_cols, inplace=True)\n\nml_df.head()\n"

In [63]:
# Instantiate LabelEncoder
from sklearn import preprocessing
le = preprocessing.LabelEncoder()

# Label Encode 'position'
ml_df['position'] = le.fit_transform(ml_df[['position']])

In [64]:
"""
# OneHotEncode 'name'
onehot_name = pd.get_dummies(ml_df['name'])
ml_df = ml_df.join(onehot_name)
ml_df = ml_df.drop('name', axis=1)

ml_df.head()
"""

"\n# OneHotEncode 'name'\nonehot_name = pd.get_dummies(ml_df['name'])\nml_df = ml_df.join(onehot_name)\nml_df = ml_df.drop('name', axis=1)\n\nml_df.head()\n"

In [65]:
# Separating features the ML models
features = ml_df.drop(labels= "Points per Game", axis=1)
features.head()

,playerId,position,icetime,% Games Played,Shots on Goal per Game,Goals per Game,Shots on Goal per Game
8171,8476878,2,0.383599,0.878049,0.256231,0.125000,0.256231
27636,8476878,2,0.343013,0.682927,0.322430,0.178571,0.322430
22871,8476878,0,0.394570,0.841463,0.238927,0.173913,0.238927
35606,8476878,0,0.405797,0.878049,0.212617,0.069444,0.212617
191,8476878,0,0.403279,0.865854,0.312360,0.098592,0.312360


In [66]:
# Separating labels the ML models
labels = ml_df["Points per Game"]
labels.head()

8171     0.222222
27636    0.321429
22871    0.275362
35606    0.250000
191      0.211268
Name: Points per Game, dtype: float64

In [67]:
# Splitting the data into training and testing datasets
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(features, labels, test_size = 0.3, random_state = 10)

## Machine Learning

Predicting points per game by player using multiple machine learning models.

### Linear Regression

In [78]:
# Splitting the data into training, validation, and testing datasets
from sklearn.model_selection import train_test_split
X_train, X_temp, y_train, y_temp = train_test_split(features, labels, test_size = 0.2, random_state = 100)
X_val, X_test, y_val, y_test = train_test_split(X_temp, y_temp, test_size = 0.5, random_state = 100)

# Train a LinearRegression model
from sklearn.metrics import log_loss
from sklearn.linear_model import LinearRegression

lm = LinearRegression()
lm.fit(X_train, y_train)
print("** Linear Regression model has been trained.")
print()  
# Get the train, validation, and test accuracy scores
print(f"** Training Score: {lm.score(X_train, y_train)}")
print(f"** Validation Score: {lm.score(X_val, y_val)}")
print(f"** Testing Score: {lm.score(X_test, y_test)}")
print()
# Make predictions
y_train_pred = lm.predict(X_train)
y_val_pred = lm.predict(X_val)
y_test_pred = lm.predict(X_test)

# Calculate evaluation metrics
mse_train = mean_squared_error(y_train, y_train_pred)
mse_val = mean_squared_error(y_val, y_val_pred)
mse_test = mean_squared_error(y_test, y_test_pred)

mae_train = mean_absolute_error(y_train, y_train_pred)
mae_val = mean_absolute_error(y_val, y_val_pred)
mae_test = mean_absolute_error(y_test, y_test_pred)

rmse_train = np.sqrt(mse_train)
rmse_val = np.sqrt(mse_val)
rmse_test = np.sqrt(mse_test)

# Print evaluation metrics
print(f"** Training MSE: {mse_train}")
print(f"** Validation MSE: {mse_val}")
print(f"** Testing MSE: {mse_test}")
print()
print(f"** Training MAE: {mae_train}")
print(f"** Validation MAE: {mae_val}")
print(f"** Testing MAE: {mae_test}")
print()
print(f"** Training RMSE: {rmse_train}")
print(f"** Validation RMSE: {rmse_val}")
print(f"** Testing RMSE: {rmse_test}")

** Linear Regression model has been trained.

** Training Score: 0.7990557428775076
** Validation Score: 0.7835763507597469
** Testing Score: 0.758515484621977

** Training MSE: 0.017068069468822954
** Validation MSE: 0.021805473441111552
** Testing MSE: 0.0210249178949475

** Training MAE: 0.09409043479297084
** Validation MAE: 0.0965019240554227
** Testing MAE: 0.09009308632169018

** Training RMSE: 0.13064482182169698
** Validation RMSE: 0.14766676484947977
** Testing RMSE: 0.1449997168788529


In [83]:
from sklearn.linear_model import SGDRegressor
from sklearn.metrics import mean_squared_error, mean_absolute_error
import numpy as np

# Splitting the data into training, validation, and testing datasets
from sklearn.model_selection import train_test_split
X_train, X_temp, y_train, y_temp = train_test_split(features, labels, test_size = 0.2, random_state = 100)
X_val, X_test, y_val, y_test = train_test_split(X_temp, y_temp, test_size = 0.5, random_state = 100)

# Set parameters for mini-batch gradient descent
batch_size = 64
n_epochs = 15000
n_batches = len(X_train) // batch_size

# Instantiate the SGDRegressor
sgd_reg = SGDRegressor(learning_rate='constant', eta0=0.001)

# Train the model using mini-batch gradient descent
for epoch in range(n_epochs):
    for i in range(n_batches):
        X_batch = X_train[i * batch_size: (i + 1) * batch_size]
        y_batch = y_train[i * batch_size: (i + 1) * batch_size]
        sgd_reg.partial_fit(X_batch, y_batch)
        
# Get the train, validation, and test accuracy scores
print(f"** Training Score: {sgd_reg.score(X_train, y_train)}")
print(f"** Validation Score: {sgd_reg.score(X_val, y_val)}")
print(f"** Testing Score: {sgd_reg.score(X_test, y_test)}")
print()

# Make predictions
y_train_pred = sgd_reg.predict(X_train)
y_val_pred = sgd_reg.predict(X_val)
y_test_pred = sgd_reg.predict(X_test)

# Calculate evaluation metrics
mse_train = mean_squared_error(y_train, y_train_pred)
mse_val = mean_squared_error(y_val, y_val_pred)
mse_test = mean_squared_error(y_test, y_test_pred)

mae_train = mean_absolute_error(y_train, y_train_pred)
mae_val = mean_absolute_error(y_val, y_val_pred)
mae_test = mean_absolute_error(y_test, y_test_pred)

rmse_train = np.sqrt(mse_train)
rmse_val = np.sqrt(mse_val)
rmse_test = np.sqrt(mse_test)

# Print evaluation metrics
print(f"** Training MSE: {mse_train}")
print(f"** Validation MSE: {mse_val}")
print(f"** Testing MSE: {mse_test}")
print()
print(f"** Training MAE: {mae_train}")
print(f"** Validation MAE: {mae_val}")
print(f"** Testing MAE: {mae_test}")
print()
print(f"** Training RMSE: {rmse_train}")
print(f"** Validation RMSE: {rmse_val}")
print(f"** Testing RMSE: {rmse_test}")

** Training Score: -4.049355319211574e+46
** Validation Score: -3.413588430396836e+46
** Testing Score: -3.950650473825888e+46

** Training MSE: 3.4394950560900923e+45
** Validation MSE: 3.439315071121144e+45
** Testing MSE: 3.439645051932139e+45

** Training MAE: 5.864720572921279e+22
** Validation MAE: 5.864567149517893e+22
** Testing MAE: 5.864848446559147e+22

** Training RMSE: 5.864720842538111e+22
** Validation RMSE: 5.864567393355749e+22
** Testing RMSE: 5.864848720923788e+22


### Linear Regression - Predictions

In [72]:
name = "Mark Scheifele"

fn = df[df['name'].str.contains(name, case = False)]
fn.head(15)

,playerId,season,name,team,position,situation,games_played,icetime,shifts,gameScore,onIce_xGoalsPercentage,offIce_xGoalsPercentage,onIce_corsiPercentage,offIce_corsiPercentage,onIce_fenwickPercentage,offIce_fenwickPercentage,iceTimeRank,I_F_xOnGoal,I_F_xGoals,I_F_xRebounds,I_F_xFreeze,I_F_xPlayStopped,I_F_xPlayContinuedInZone,I_F_xPlayContinuedOutsideZone,I_F_flurryAdjustedxGoals,I_F_scoreVenueAdjustedxGoals,I_F_flurryScoreVenueAdjustedxGoals,I_F_primaryAssists,I_F_secondaryAssists,I_F_shotsOnGoal,I_F_missedShots,I_F_blockedShotAttempts,I_F_shotAttempts,I_F_points,I_F_goals,I_F_rebounds,I_F_reboundGoals,I_F_freeze,I_F_playStopped,I_F_playContinuedInZone,I_F_playContinuedOutsideZone,I_F_savedShotsOnGoal,I_F_savedUnblockedShotAttempts,penalties,I_F_penalityMinutes,I_F_faceOffsWon,I_F_hits,I_F_takeaways,I_F_giveaways,I_F_lowDangerShots,I_F_mediumDangerShots,I_F_highDangerShots,I_F_lowDangerxGoals,I_F_mediumDangerxGoals,I_F_highDangerxGoals,I_F_lowDangerGoals,I_F_mediumDangerGoals,I_F_highDangerGoals,I_F_scoreAdjustedShotsAttempts,I_F_unblockedShotAttempts,I_F_scoreAdjustedUnblockedShotAttempts,I_F_dZoneGiveaways,I_F_xGoalsFromxReboundsOfShots,I_F_xGoalsFromActualReboundsOfShots,I_F_reboundxGoals,I_F_xGoals_with_earned_rebounds,I_F_xGoals_with_earned_rebounds_scoreAdjusted,I_F_xGoals_with_earned_rebounds_scoreFlurryAdjusted,I_F_shifts,I_F_oZoneShiftStarts,I_F_dZoneShiftStarts,I_F_neutralZoneShiftStarts,I_F_flyShiftStarts,I_F_oZoneShiftEnds,I_F_dZoneShiftEnds,I_F_neutralZoneShiftEnds,I_F_flyShiftEnds,faceoffsWon,faceoffsLost,timeOnBench,penalityMinutes,penalityMinutesDrawn,penaltiesDrawn,shotsBlockedByPlayer,OnIce_F_xOnGoal,OnIce_F_xGoals,OnIce_F_flurryAdjustedxGoals,OnIce_F_scoreVenueAdjustedxGoals,OnIce_F_flurryScoreVenueAdjustedxGoals,OnIce_F_shotsOnGoal,OnIce_F_missedShots,OnIce_F_blockedShotAttempts,OnIce_F_shotAttempts,OnIce_F_goals,OnIce_F_rebounds,OnIce_F_reboundGoals,OnIce_F_lowDangerShots,OnIce_F_mediumDangerShots,OnIce_F_highDangerShots,OnIce_F_lowDangerxGoals,OnIce_F_mediumDangerxGoals,OnIce_F_highDangerxGoals,OnIce_F_lowDangerGoals,OnIce_F_mediumDangerGoals,OnIce_F_highDangerGoals,OnIce_F_scoreAdjustedShotsAttempts,OnIce_F_unblockedShotAttempts,OnIce_F_scoreAdjustedUnblockedShotAttempts,OnIce_F_xGoalsFromxReboundsOfShots,OnIce_F_xGoalsFromActualReboundsOfShots,OnIce_F_reboundxGoals,OnIce_F_xGoals_with_earned_rebounds,OnIce_F_xGoals_with_earned_rebounds_scoreAdjusted,OnIce_F_xGoals_with_earned_rebounds_scoreFlurryAdjusted,OnIce_A_xOnGoal,OnIce_A_xGoals,OnIce_A_flurryAdjustedxGoals,OnIce_A_scoreVenueAdjustedxGoals,OnIce_A_flurryScoreVenueAdjustedxGoals,OnIce_A_shotsOnGoal,OnIce_A_missedShots,OnIce_A_blockedShotAttempts,OnIce_A_shotAttempts,OnIce_A_goals,OnIce_A_rebounds,OnIce_A_reboundGoals,OnIce_A_lowDangerShots,OnIce_A_mediumDangerShots,OnIce_A_highDangerShots,OnIce_A_lowDangerxGoals,OnIce_A_mediumDangerxGoals,OnIce_A_highDangerxGoals,OnIce_A_lowDangerGoals,OnIce_A_mediumDangerGoals,OnIce_A_highDangerGoals,OnIce_A_scoreAdjustedShotsAttempts,OnIce_A_unblockedShotAttempts,OnIce_A_scoreAdjustedUnblockedShotAttempts,OnIce_A_xGoalsFromxReboundsOfShots,OnIce_A_xGoalsFromActualReboundsOfShots,OnIce_A_reboundxGoals,OnIce_A_xGoals_with_earned_rebounds,OnIce_A_xGoals_with_earned_rebounds_scoreAdjusted,OnIce_A_xGoals_with_earned_rebounds_scoreFlurryAdjusted,OffIce_F_xGoals,OffIce_A_xGoals,OffIce_F_shotAttempts,OffIce_A_shotAttempts,xGoalsForAfterShifts,xGoalsAgainstAfterShifts,corsiForAfterShifts,corsiAgainstAfterShifts,fenwickForAfterShifts,fenwickAgainstAfterShifts,Last_Season_Played,% Games Played,Points per Game,Goals per Game,Shots on Goal per Game
8656,8476460,2022,Mark Scheifele,WPG,C,all,76,93560.0,1681.0,63.51,0.56,0.49,0.58,0.48,0.57,0.48,149.0,201.62,29.95,16.15,39.23,6.75,116.49,73.42,28.56,29.97,28.58,13.0,11.0,193.0,89.0,58.0,340.0,63.0,39.0,14.0,3.0,34.0,4.0,98.0,93.0,154.0,243.0,10.0,31.0,560.0,48.0,58.0,41.0,148.0,105.0,29.0,5.71,13.11,11.13,10.0,18.0,11.0,340.27,282.0,282.01,15.0,3.83,2.64,3.81,29.97,30.03,28.77

In [73]:
# Predict
pred = lm.predict([[8476460, .38, .80, 0.2, .25, 0.5, .22]])
print(name + " will average:", float(np.round(pred, 4)), "points per game.")

Mark Scheifele will average: 1.2127 points per game.


### Neural Network